In [1]:
import DualwormFunctions as dw
import Observables as obs
%load_ext line_profiler
%load_ext memory_profiler
import RunBasis
import argparse
import StartStates as strst
import numpy as np

In [13]:
nst = 50;
nsm = 256;
nips = 30;
nb = 128;
measperiod = 2;

In [2]:
def genArgs():
    parser = argparse.ArgumentParser()
    
    parser.add_argument('-f', type = str)

    parser.add_argument('--L', type = int, default = 12, help = 'Lattice side size')

    # COUPLINGS
    parser.add_argument('--J1', type = float, default = 1,
                        help = 'NN coupling') # nearest-neighbour coupling
    parser.add_argument('--J2', type = float, default = 0.0,
                        help = '2nd NN coupling') # 2nd NN coupling
    parser.add_argument('--J3', type = float, default = 0.0,
                        help = '3rd NN coupling') # 3rd NN coupling
    parser.add_argument('--J4', type = float, default = 0.0,
                        help = '4th NN coupling')
    parser.add_argument('--h', type = float, default = measperiod,
                        help = 'Magnetic field')    
    #NUMBER OF STEPS AND ITERATIONS
    parser.add_argument('--nst', type = int, default = nst,
                        help = 'number of thermalisation steps') # number of thermalisation steps
    parser.add_argument('--nsm', type = int, default = nsm,
                        help = 'number of measurements steps') # number of measurement steps
    parser.add_argument('--nips', type = int, default = nips,
                        help = 'number of worm constructions per MC step')
    parser.add_argument('--measperiod', type = int, default = 2,
                        help = 'number of nips worm building + swaps between measurements')
    parser.add_argument('--nb', type = int, default = nb,
                        help = 'number of bins')

    #PARALLELISATION
    parser.add_argument('--ncores', type = int, default = 8,
                        help = 'number of threads to use')

    #WORM PARAMETERS
    parser.add_argument('--nmaxiter', type = int, default = 10,
                        help = '''maximal number of segments in a loop update over the
                        size of the lattice (1 = 1times the number of dualbonds in the
                        lattice)''')
    parser.add_argument('--randominit', default = False, action ='store_true',
                        help = 'intialise the states randomly')
    parser.add_argument('--same', default = False, action = 'store_true',
                        help = '''initialise all temperatures with the same
                        state (debug purposes)''')
    parser.add_argument('--magninit', default = False, action = 'store_true',
                        help = '''initialise all the temperature with the maximally magnetised GS''')
    parser.add_argument('--measupdate', default = False, action = 'store_true',
                       help = '''activate to mimic the action of the measuring tip''')
    parser.add_argument('--p', type = float, default = 0.0, 
                       help = '''prob of the measuring tip flipping the spin (number between 0 and 1)''')
    parser.add_argument('--ssf', default = True, action = 'store_true',
                        help = 'activate for single spin flip update')
    #TEMPERATURE PARAMETERS
    parser.add_argument('--t_list', nargs = '+', type = float, default = [0.05,20],
                        help = 'list of limiting temperature values')
    parser.add_argument('--nt_list', nargs = '+', type = int, default = [84],
                        help = 'list of number of temperatures in between the given limiting temperatures')
    parser.add_argument('--log_tlist', default = True, action='store_true',
                        help = 'state whether you want the temperature be spaced log-like or linear-like (activate if you want log)')
    parser.add_argument('--stat_temps_lims', nargs = '+', type = float,
                        help = '''limiting temperatures for the various ranges of
                        measurements''') 
                        #default will be set to none, and then we can decide what to do later on.
    
    #CORRELATIONS PARAMETER
    parser.add_argument('--energy', default = True, action = 'store_true',
                        help = 'activate if you want to save the energy')
    parser.add_argument('--magnetisation', default = True, action = 'store_true',
                        help = 'activate if you want to save the magnetisation')
    parser.add_argument('--magnstats', default = False, action = 'store_true', 
                       help = 'activate if you want to compute the magnetisation statistics')
    parser.add_argument('--charges', default = True, action = 'store_true',
                        help = 'activate if you want to save the charges')
    parser.add_argument('--correlations', default = True, action = 'store_true',
                        help = 'activate if you want to save either central or all correlations')
    parser.add_argument('--all_correlations', default = False, action = 'store_true',
                        help = '''activate if you want to save the correlations for all non-equivalent
                        pairs of sites. Otherwise, will save central correlations.''')
    parser.add_argument('--firstcorrelations', default = False, action = 'store_true',
                        help = 'activate if you want to save first correlations, otherwise will save central')
    #SAVE
    parser.add_argument('--output', type = str, default = "test", help = 'saving filename (.pkl will be added)')
    args = parser.parse_args()
    return args

In [3]:
def callRunBasis():
    args = genArgs()
    return RunBasis.main(args)

In [4]:
meanstat, failedupdatesth, failedupdates = callRunBasis() 

Lattice side size:  12
J1  1
J2  0.0
J3  0.0
J3st  0.0
h 1
Couplings exacted
dw.Hamiltonian done
hamiltonian computed
0 84
Number of temperatures:  84
Fully random initialisation =  False
Identical initialisation =  False
Magnetisation initialisation =  False
{'random': False, 'same': False, 'magninit': False, 'h': 1}
Same initialisation for all temperatures =  False
False
not random
Initialisation type:  J1
J1Init
List of measurements to be performed: ['Energy', 'Magnetisation', 'Charges', 'Si', 'Central_Correlations']
List of temperature indices to measure: range(0, 84)
-----------Thermalisation------------------
Number of thermalisation steps =  50
itermcs =  50
magnstats False
statsfunctions []
h =  1
Time for mcsevolve = 0.12775915145874023
Time for tempering = 0.0014353609085083006
Time for mapping to spins + computing statistics= 2.632141113281251e-06
Time for all thermalisation steps =  6.460557699203491
-----------Measurements-----------------
Number of measurement steps =  25

In [9]:
print(failedupdatesth/(nst*nips))

[0.99989763 0.99989969 0.99990586 0.99990381 0.99990123 0.99989352
 0.99990689 0.99990535 0.99990278 0.99990741 0.99989198 0.99990123
 0.99989198 0.99990226 0.99990586 0.99989249 0.99990638 0.99989043
 0.99986111 0.99983436 0.99979784 0.99966152 0.99953344 0.99922634
 0.9987716  0.99805144 0.9969892  0.99546142 0.99338529 0.99028807
 0.98659414 0.98152263 0.97496862 0.96704115 0.95734877 0.9460571
 0.93313529 0.91844136 0.90167953 0.88454115 0.86463632 0.84414352
 0.82236831 0.7988714  0.77507202 0.7497392  0.72555556 0.69978241
 0.67440329 0.64855144 0.62284362 0.59558179 0.57074743 0.54546656
 0.52019187 0.49552109 0.47127212 0.4483071  0.42561471 0.40299486
 0.38219907 0.36068416 0.34162243 0.32231224 0.30413169 0.28641409
 0.26995062 0.25413735 0.23882819 0.22429475 0.21094496 0.1984285
 0.18550874 0.17467078 0.16348457 0.15312809 0.1433287  0.13391049
 0.12561883 0.11714558 0.10991924 0.10298097 0.09593673 0.08903138]


In [14]:
print(failedupdates/(nsm*nips*measperiod))

[0.99999985 1.         0.9999998  0.99999965 0.9999999  1.
 1.         0.9999999  0.9999999  0.9999999  0.9999999  0.99999995
 0.99999985 0.99999995 0.99999975 0.99999839 0.99999473 0.99998799
 0.99997674 0.99994821 0.99989606 0.99980564 0.999625   0.99934022
 0.99886078 0.99814292 0.99707951 0.99555513 0.99337846 0.99040236
 0.98644542 0.98140819 0.9750205  0.96701725 0.95754224 0.94633674
 0.93332314 0.91851224 0.90222012 0.88408088 0.86499476 0.84398515
 0.82182768 0.79879552 0.77479765 0.75037616 0.72523756 0.70007369
 0.6743695  0.64810871 0.62226075 0.5959863  0.5705587  0.54519721
 0.52020329 0.4956332  0.47178729 0.44817266 0.42534813 0.40326797
 0.381914   0.36112    0.34123912 0.32216661 0.30356124 0.28624343
 0.26962665 0.25412241 0.23909731 0.22430194 0.21069753 0.19793214
 0.18563835 0.17415671 0.16313733 0.15289472 0.14323545 0.13380584
 0.12536365 0.11704046 0.10947165 0.10205003 0.09544597 0.08905984]
